In [ ]:
# ===================================================================
# Cellule 1 : Import des bibliothèques et configuration initiale
# ===================================================================
import os
import gc
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psutil
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
from io import StringIO
import json
from datetime import datetime

# Import de PyTorch Geometric
try:
    from torch_geometric.data import Data
    from torch_geometric.nn import GCNConv, VGAE
    from torch_geometric.utils import train_test_split_edges, negative_sampling
    import torch.nn.functional as F
except ImportError:
    print("ERREUR: PyTorch Geometric n'est pas installé.")
    print("Veuillez l'installer, par exemple avec : pip install torch-geometric")
    # Pour Colab, des commandes plus spécifiques peuvent être nécessaires.
    exit()

# Configuration de l'affichage et des visualisations
%matplotlib inline
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

# Définition de l'appareil (GPU si disponible, sinon CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilisation de l'appareil : {device}")

In [ ]:
# ===================================================================
# Cellule 2 : Chargement et préparation des nouvelles données
# ===================================================================
# MODIFICATION : Noms des fichiers de la version 2
graph_path = 'construction/credential_stuffing_graph_v2.pt'
mapping_path = 'construction/node_mapping_v2.pt'

if not os.path.exists(graph_path) or not os.path.exists(mapping_path):
    raise FileNotFoundError(
        "Fichiers de graphe V2 non trouvés. "
        "Assurez-vous d'avoir exécuté 'construction_graph.ipynb' avec les nouvelles données."
    )

# Charger les objets PyG et le mapping
data = torch.load(graph_path, weights_only=False)
idx_to_node = torch.load(mapping_path, weights_only=False)
node_to_idx = {v: k for k, v in idx_to_node.items()}

print("--- Graphe V2 chargé ---")
print(data)
print(f"Nombre de caractéristiques par nœud : {data.num_node_features}")

# --- Normalisation des caractéristiques des nœuds (Z-score) ---
# C'est une étape cruciale pour la performance des modèles de deep learning.
x_mean = data.x.mean(dim=0, keepdim=True)
x_std = data.x.std(dim=0, keepdim=True) + 1e-8  # Ajouter epsilon pour éviter division par 0
data.x = (data.x - x_mean) / x_std
print("\nCaractéristiques des nœuds normalisées.")

# Déplacer les données vers le bon appareil
data = data.to(device)
print(f"Données déplacées vers '{device}'.")

# --- Reconstruction d'un DataFrame de caractéristiques pour l'analyse ---
# Cela nous permettra de consulter facilement les caractéristiques d'un nœud (ex: failure_rate).
feature_names = [
    'is_ip', 'is_user', 'degree', 'total_attempts', 
    'failure_rate', 'specific_feature' # ('unique_users' ou 'unique_ips')
]
node_features_df = pd.DataFrame(data.x.cpu().numpy(), columns=feature_names)
node_features_df['node_id'] = node_features_df.index.map(idx_to_node.get)
node_features_df = node_features_df.set_index('node_id')

print("\nAperçu du DataFrame des caractéristiques des nœuds :")
display(node_features_df.head())

In [ ]:
# ===================================================================
# Cellule 3 : Définition des classes de modèles (VGAE) 
# ===================================================================

# --- 1. Encodeur de base ---
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, cached=True)
        self.conv_mu = GCNConv(hidden_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(hidden_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)
    

# --- 2. Encodeur amélioré ---
class ImprovedVariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, dropout=0.3):
        super().__init__()
        self.dropout = dropout
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()

        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.conv_mu = GCNConv(hidden_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(hidden_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        for i in range(len(self.convs)):
            x = self.convs[i](x, edge_index)
            x = self.bns[i](x)
            x = F.elu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)
    
# --- 3. Classe VGAE modifiée pour intégrer le facteur beta ---
class CustomVGAE(torch.nn.Module):
    def __init__(self, encoder, beta=1.0):
        super().__init__()
        self.encoder = encoder
        self.beta = beta

    def reparametrize(self, mu, logstd):
        if self.training:
            return mu + torch.randn_like(logstd) * torch.exp(logstd)
        return mu

    def decode(self, z, edge_index):
        row, col = edge_index
        dot_product = torch.sum(z[row] * z[col], dim=1)
        return torch.sigmoid(dot_product)
        
    # ====================================================================
    # CORRECTION : Ajout de la méthode decode_all manquante.
    # Cette méthode reconstruit la matrice d'adjacence complète.
    def decode_all(self, z):
        """
        Décodeur complet qui calcule la matrice d'adjacence reconstruite.
        """
        # Le produit scalaire de tous les embeddings (z) avec tous les autres (z.t())
        # donne la matrice complète des similarités.
        adj = torch.matmul(z, z.t())
        return torch.sigmoid(adj)
    # ====================================================================

    def kl_loss(self, mu, logstd):
        kl = -0.5 * torch.mean(torch.sum(1 + 2 * logstd - mu**2 - (2 * logstd).exp(), dim=1))
        return self.beta * kl

print("Classes des modèles définies (avec méthode decode_all ajoutée).")

In [ ]:
# ===================================================================
# CELLULE 4 : Fonction d'entraînement et d'évaluation du modèle
# ===================================================================
def train_model(model, train_data, epochs, lr=0.01, weight_decay=1e-5):
    """Fonction générique pour entraîner notre CustomVGAE."""
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    losses = []
    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        
        mu, logstd = model.encoder(train_data.x, train_data.train_pos_edge_index)
        z = model.reparametrize(mu, logstd)
        
        pos_edge_index = train_data.train_pos_edge_index
        pos_pred = model.decode(z, pos_edge_index)
        recon_loss = -torch.log(pos_pred + 1e-15).mean()
        
        kl_loss = (1 / train_data.num_nodes) * model.kl_loss(mu, logstd)
        
        loss = recon_loss + kl_loss
                
        loss.backward()
        
        # ====================================================================
        # CORRECTION DE STABILITÉ : Ajout du rognage de gradient
        # Cela empêche les gradients d'exploser, cause fréquente de NaN.
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # ====================================================================

        optimizer.step()
        losses.append(loss.item())
        
        if epoch % 20 == 0 or epoch == 1:
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
            
    return model, losses

# NOTE : La fonction d'évaluation doit aussi être adaptée
def evaluate_model(model, data):
    """Évalue un modèle sur les ensembles de validation et de test."""
    model.eval()
    with torch.no_grad():
        mu, logstd = model.encoder(data.x, data.train_pos_edge_index)
        z = model.reparametrize(mu, logstd)
        
        # Validation
        pos_pred_val = model.decode(z, data.val_pos_edge_index)
        neg_pred_val = model.decode(z, data.val_neg_edge_index)
        y_pred_val = torch.cat([pos_pred_val, neg_pred_val]).cpu()
        y_true_val = torch.cat([torch.ones_like(pos_pred_val), torch.zeros_like(neg_pred_val)]).cpu()
        
        # Vérification pour un débogage facile
        if torch.isnan(y_pred_val).any():
            raise ValueError("Erreur critique : NaN détecté dans les prédictions de validation !")

        val_auc = roc_auc_score(y_true_val, y_pred_val)
        val_ap = average_precision_score(y_true_val, y_pred_val)

        # Test
        pos_pred_test = model.decode(z, data.test_pos_edge_index)
        neg_pred_test = model.decode(z, data.test_neg_edge_index)
        y_pred_test = torch.cat([pos_pred_test, neg_pred_test]).cpu()
        y_true_test = torch.cat([torch.ones_like(pos_pred_test), torch.zeros_like(neg_pred_test)]).cpu()
        
        if torch.isnan(y_pred_test).any():
            raise ValueError("Erreur critique : NaN détecté dans les prédictions de test !")
            
        test_auc = roc_auc_score(y_true_test, y_pred_test)
        test_ap = average_precision_score(y_true_test, y_pred_test)
    
    return val_auc, val_ap, test_auc, test_ap

print("Fonctions d'entraînement (avec clipping) et d'évaluation prêtes.")

In [ ]:
# ===================================================================
# Cellule 5 : Instanciation, Entraînement et Évaluation 
# ===================================================================

# =================================================================================
# CORRECTION : On sauvegarde le edge_index original AVANT de le perdre.
# La fonction train_test_split_edges supprime l'attribut data.edge_index.
# On utilise .clone() pour s'assurer d'avoir une copie indépendante.
# =================================================================================
original_edge_index = data.edge_index.clone()

# --- 1. Diviser les données ---
# Cette opération est stochastique et supprime data.edge_index.
data_split = train_test_split_edges(data, val_ratio=0.05, test_ratio=0.1)
data_split = data_split.to(device)
print("Données divisées pour l'entraînement/validation/test :\n", data_split)
# À ce stade, data.edge_index est None, mais nous avons notre sauvegarde !

# --- 2. Définir les hyperparamètres ---
IN_CHANNELS = data.num_node_features
HIDDEN_CHANNELS = 64
LATENT_DIM = 32
EPOCHS = 200
LEARNING_RATE = 0.01

# --- 3. Modèle Original ---
print("\n--- Entraînement du Modèle Original ---")
encoder_orig = VariationalGCNEncoder(IN_CHANNELS, HIDDEN_CHANNELS, LATENT_DIM)
model_orig = CustomVGAE(encoder_orig, beta=1.0).to(device)
model_orig, losses_orig = train_model(model_orig, data_split, epochs=EPOCHS, lr=LEARNING_RATE)

# --- 4. Modèle Amélioré ---
print("\n--- Entraînement du Modèle Amélioré ---")
encoder_imp = ImprovedVariationalGCNEncoder(IN_CHANNELS, HIDDEN_CHANNELS * 2, LATENT_DIM * 2)
model_improved = CustomVGAE(encoder_imp, beta=0.5).to(device)
model_improved, losses_imp = train_model(model_improved, data_split, epochs=EPOCHS, lr=LEARNING_RATE)

# --- 5. Évaluation comparative ---
print("\n--- Évaluation finale ---")
val_auc_orig, val_ap_orig, test_auc_orig, test_ap_orig = evaluate_model(model_orig, data_split)
val_auc_imp, val_ap_imp, test_auc_imp, test_ap_imp = evaluate_model(model_improved, data_split)

results_df = pd.DataFrame({
    'Modèle': ['Original', 'Amélioré'],
    'Test AUC': [test_auc_orig, test_auc_imp],
    'Test AP': [test_ap_orig, test_ap_imp],
    'Val AUC': [val_auc_orig, val_auc_imp],
    'Val AP': [val_ap_orig, val_ap_imp]
})
print("\n--- RÉSULTATS COMPARATIFS ---")
print(results_df.round(4))

# --- 6. Visualisation des pertes ---
plt.figure(figsize=(12, 5))
plt.plot(losses_orig, label='Perte Modèle Original', alpha=0.8)
plt.plot(losses_imp, label='Perte Modèle Amélioré', alpha=0.8)
plt.title("Courbes d'apprentissage des modèles")
plt.xlabel("Epoch")
plt.ylabel("Perte")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ===================================================================
# Cellule 6 : Détection d'anomalies et analyse - CORRIGÉE
# ===================================================================

def get_anomaly_scores(model, data_split_obj, full_edge_index):
    """Calcule les erreurs de reconstruction pour tous les nœuds."""
    model.eval()
    with torch.no_grad():
        # L'encodage utilise la structure du graphe d'entraînement
        mu, _ = model.encoder(data_split_obj.x, data_split_obj.train_pos_edge_index)
        z = mu
        
        # Calculer l'erreur de reconstruction pour chaque arête potentielle
        recon_error_matrix = 1 - model.decode_all(z)
        
        # Agréger l'erreur par nœud en utilisant les arêtes du graphe COMPLET
        node_errors = torch.zeros(data_split_obj.num_nodes, device=device)
        
        # On utilise le `full_edge_index` passé en argument
        row, col = full_edge_index
        
        edge_errors = recon_error_matrix[row, col]
        node_errors.scatter_add_(0, row, edge_errors)
        node_errors.scatter_add_(0, col, edge_errors)
        
        # Normaliser par le degré du nœud dans le graphe complet
        degrees = torch.bincount(row, minlength=data_split_obj.num_nodes) + torch.bincount(col, minlength=data_split_obj.num_nodes)
        node_errors /= degrees.clamp(min=1)
        
    return node_errors.cpu().numpy()

# =================================================================================
# CORRECTION : On appelle la fonction en passant notre sauvegarde `original_edge_index`
# que nous avons créée dans la cellule précédente.
# =================================================================================
scores_orig = get_anomaly_scores(model_orig, data_split, original_edge_index)
scores_imp = get_anomaly_scores(model_improved, data_split, original_edge_index)

# ... (le reste de la cellule est inchangé)
# Créer un DataFrame avec les scores et les caractéristiques
anomaly_df = node_features_df.copy()
anomaly_df['score_orig'] = scores_orig
anomaly_df['score_imp'] = scores_imp

# Identifier les nœuds de type IP
ip_anomaly_df = anomaly_df[anomaly_df['is_ip'] > 0.5].copy()

# Afficher les IPs les plus suspectes selon chaque modèle
print("\n--- Top 10 IPs suspectes (Modèle Original) ---")
display(ip_anomaly_df.sort_values('score_orig', ascending=False).head(10))

print("\n--- Top 10 IPs suspectes (Modèle Amélioré) ---")
display(ip_anomaly_df.sort_values('score_imp', ascending=False).head(10))

# Visualiser la distribution des scores d'anomalie
plt.figure(figsize=(14, 6))
sns.kdeplot(ip_anomaly_df['score_orig'], label='Scores Modèle Original', fill=True)
sns.kdeplot(ip_anomaly_df['score_imp'], label='Scores Modèle Amélioré', fill=True)
plt.title("Distribution des Scores d'Anomalie pour les IPs")
plt.xlabel("Score d'Anomalie (Erreur de Reconstruction)")
plt.legend()
plt.show()

In [ ]:
# ===================================================================
# Cellule 7 : Visualisation T-SNE des Espaces Latents - CORRIGÉE
# ===================================================================

# =================================================================================
# CORRECTION : La fonction doit accepter les mêmes arguments que get_anomaly_scores
# pour avoir accès aux bonnes listes d'arêtes.
# =================================================================================
def visualize_tsne(model, data_split_obj, full_edge_index, title):
    """Génère et affiche une visualisation t-SNE de l'espace latent."""
    model.eval()
    with torch.no_grad():
        # =================================================================================
        # CORRECTION :
        # 1. Utiliser model.encoder au lieu de model.encode.
        # 2. Récupérer mu comme la représentation latente (z).
        # 3. Utiliser les arêtes d'entraînement (train_pos_edge_index) pour l'encodage.
        # =================================================================================
        mu, _ = model.encoder(data_split_obj.x, data_split_obj.train_pos_edge_index)
        z = mu.cpu().numpy()
        
    # Appliquer t-SNE
    print(f"Calcul de t-SNE pour '{title}'...")
    tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
    z_tsne = tsne.fit_transform(z)
    
    # Séparer IPs et Utilisateurs pour la visualisation
    is_ip = data_split_obj.x[:, 0].cpu().numpy() > 0.5
    
    plt.figure(figsize=(10, 10))
    # Afficher les utilisateurs en premier (en arrière-plan)
    plt.scatter(z_tsne[~is_ip, 0], z_tsne[~is_ip, 1], s=20, color='gray', alpha=0.5, label='Utilisateurs')
    
    # =================================================================================
    # CORRECTION : L'appel interne à get_anomaly_scores doit aussi utiliser
    # les bons arguments que nous avons maintenant à disposition.
    # =================================================================================
    ip_scores = get_anomaly_scores(model, data_split_obj, full_edge_index)[is_ip]
    plt.scatter(z_tsne[is_ip, 0], z_tsne[is_ip, 1], s=50, c=ip_scores, cmap='Reds', alpha=0.9, label='IPs')
    
    plt.title(title)
    plt.xlabel("Composante t-SNE 1")
    plt.ylabel("Composante t-SNE 2")
    plt.legend()
    cbar = plt.colorbar()
    cbar.set_label("Score d'Anomalie")
    plt.show()

# =================================================================================
# CORRECTION : On appelle la fonction de visualisation en passant les bons objets :
# `data_split` et la sauvegarde `original_edge_index`.
# =================================================================================
visualize_tsne(model_orig, data_split, original_edge_index, "Espace Latent t-SNE - Modèle Original")
visualize_tsne(model_improved, data_split, original_edge_index, "Espace Latent t-SNE - Modèle Amélioré")

In [ ]:
# ===================================================================
# Cellule 8 : Sauvegarde des résultats et du meilleur modèle
# ===================================================================
results_dir = "results_v2"
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# --- 1. Sauvegarder le meilleur modèle (basé sur le Test AP) ---
best_model = model_improved if test_ap_imp > test_ap_orig else model_orig
model_name = "improved_vgae" if test_ap_imp > test_ap_orig else "original_vgae"
model_path = os.path.join(results_dir, f"best_model_{model_name}.pt")
torch.save(best_model.state_dict(), model_path)
print(f"Meilleur modèle ('{model_name}') sauvegardé dans : {model_path}")

# --- 2. Sauvegarder le tableau de comparaison ---
comparison_path = os.path.join(results_dir, "model_comparison.csv")
results_df.to_csv(comparison_path, index=False)
print(f"Tableau de comparaison sauvegardé dans : {comparison_path}")

# --- 3. Sauvegarder la liste des IPs les plus suspectes ---
top_anomalies_path = os.path.join(results_dir, "top_anomalies.csv")
ip_anomaly_df.sort_values('score_imp', ascending=False).head(100).to_csv(top_anomalies_path)
print(f"Top 100 des IPs suspectes sauvegardées dans : {top_anomalies_path}")